In [29]:
import pandas as pd

# d_list = ['180716', '180717', '180718', '180719', '180720', \
#           '180721', '180722', '180723', '180724', '180725', \
#           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804',\
#           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
#           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822',\
d_list = ['180827', '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
          '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
          '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925',\
          '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005',\
          '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015',\
          '181016', '181017', '181018', '181019', '181020', '181021',\
          '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030',\
           '181031', '181101', '181102', '181103', '181104', '181105', '181106', '181107', '181108', '181109',\
           '181110', '181111']
pred_day = ['181105', '181106', '181107', '181108', '181109','181110', '181111']

In [20]:
import math
def my_pow(p,n):
    if p>1:
        return 1
    else:
        return math.pow(p,n)
def get_avg_run_stat_mint(s):
    # 把时间转换成10进制数据  60进制-->100进制
    if s is None:
        return None
    else:
        h = s//10000
        m = (s%10000)//100
        
        s = s%100 #不用精确到秒
        return h*60+m+s/60

In [54]:
#资源池总量；多个文件合并。

# res1 = pd.read_csv(r'E:\jupyter\CTM\data_src\resorce\res_0715.csv')
res2 = pd.read_csv(r'E:\jupyter\CTM\data_src\resorce\res_0825.csv')
res3 = pd.read_csv(r'E:\jupyter\CTM\data_src\resorce\res_1020.csv')
res4 = pd.read_csv(r'E:\jupyter\CTM\data_src\resorce\res_1111.csv')
res5 = pd.read_csv(r'E:\jupyter\CTM\data_src\resorce\res_1111_.csv')
res = pd.concat([res2,res3,res4,res5], ignore_index=True)

print(res.shape)
res = res[res['D'] == 'D']
print(res.shape)
res.drop_duplicates(subset=['resource','v_day','server'],keep='first',inplace=True)
print(res.shape)
res['v_day'] = res['v_day'].apply(str).apply(lambda x:x[:7])
res['v_day'] = res['v_day'].apply(lambda x:x.replace('\x1c',''))  #最后几天的v_day有特殊字符,需要过滤掉

(280604, 5)
(247598, 5)
(222226, 5)


In [55]:


for day in pred_day:
#for day in d_list:
# for day in ['181018']:
    #day = '180603'   #调试用

    d_res = res[res.v_day == day]
    
    #---------------------------------------------------------------------------------------------------
    ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob_0\ajob_%s_0.csv'%day,low_memory=False)
    ajob = ajob[['order_id','job_name','avg_start_time']]
    #-----------------------------------------------------------------------------------------------------
    
    if day in pred_day:
        d_job_res = pd.read_csv(r'E:\jupyter\CTM\data_src\resorce\job_res_num_%s_.csv'%day)
    else:
        d_job_res = pd.read_csv(r'E:\jupyter\CTM\data_src\resorce\job_res_num_%s.csv'%day)
        
    d_job_res.drop_duplicates(subset=['order_id','resource','server'],keep='last',inplace=True)

    
    d_job_res = d_job_res.merge(d_res[['resource','server','max_res']],on =['resource','server'],how='left')

    #构建唯一键
    d_job_res['server'] = d_job_res['server'].apply(str)
    d_job_res['res_key'] = d_job_res[['resource','server']].apply(lambda x:x[1]+x[0],axis=1)
    d_job_res['order_id'] = d_job_res[['order_id','server']].apply(lambda x:x[0]+x[1],axis=1)

    #----------------------------------------------------------------------------------------------------
    #去掉其他天实例化的资源
    #print(d_job_res.shape)
    d_job_res = d_job_res.merge(ajob,how='left',on='order_id')
    d_job_res = d_job_res[d_job_res.job_name.notnull()]
    #print(d_job_res.shape)
    
    del d_job_res['job_name']
    
    #计算同一资源，有多少其他作业会争抢
    res_groupby = d_job_res.groupby(['res_key']).quant.sum()
    res_groupby = res_groupby.to_frame(name='res_quat_sum')
    res_groupby['res_key'] = res_groupby.index

    #计算同一资源，有多少其他作业会争抢
    res_groupby_avgstart = d_job_res.groupby(['res_key']).avg_start_time.std()
    res_groupby_avgstart = res_groupby_avgstart.to_frame(name='res_avg_start_std')
    res_groupby_avgstart['res_key'] = res_groupby_avgstart.index
    
    #合并到同一个表里。
    d_job_res = d_job_res.merge(res_groupby,on = 'res_key',how='left')
    d_job_res = d_job_res.merge(res_groupby_avgstart,on = 'res_key',how='left')
    
    del d_job_res['resource']
    del d_job_res['server']

    
    # 计算大致概率
    d_job_res['p'] = d_job_res[['max_res','res_quat_sum']].apply(lambda x:x[0]/x[1] ,axis=1)
    d_job_res['p_pow'] = d_job_res[['p','quant']].apply(lambda x:my_pow(x[0],x[1]),axis=1)
    
    #概率  考虑作业开始时间
    d_job_res['p_div_std'] = d_job_res[['p','res_avg_start_std']].apply(lambda x:x[0]*x[1] ,axis=1)
    d_job_res['p_pow_div_std'] = d_job_res[['p_pow','res_avg_start_std']].apply(lambda x:x[0]*x[1],axis=1)    

    #将每个作业的所有资源概率累乘
    sigle_job_res_prod = d_job_res.groupby('order_id')['p_pow'].prod()
    sigle_job_res_prod = sigle_job_res_prod.to_frame(name='p_prod')
    sigle_job_res_prod['order_id'] = sigle_job_res_prod.index
    sigle_job_res_prod['p_prod'] = sigle_job_res_prod['p_prod'].apply(lambda x:x*100)


    #sigle_job_res_prod = pd.read_csv(r'E:\jupyter\CTM\data_processed\resorce\p_prod_%s.csv'%day)
    
    #----------------------------------------------------------------------------------
    sigle_job_res_prod_min = d_job_res.groupby('order_id')['p_pow'].min()
    sigle_job_res_prod_min = sigle_job_res_prod_min.to_frame(name='p_prod_min')
    sigle_job_res_prod_min['order_id'] = sigle_job_res_prod_min.index
    
    sigle_job_res_prod_max = d_job_res.groupby('order_id')['p_pow'].max()
    sigle_job_res_prod_max = sigle_job_res_prod_max.to_frame(name='p_prod_max')
    sigle_job_res_prod_max['order_id'] = sigle_job_res_prod_max.index
    
    job_rese_prod_std_min = d_job_res.groupby('order_id')['p_div_std'].min()
    job_rese_prod_std_min = job_rese_prod_std_min.to_frame(name='p_div_std_min')
    job_rese_prod_std_min['order_id'] = job_rese_prod_std_min.index
    
    job_rese_prod_std_max = d_job_res.groupby('order_id')['p_div_std'].max()
    job_rese_prod_std_max = job_rese_prod_std_max.to_frame(name='p_div_std_max')
    job_rese_prod_std_max['order_id'] = job_rese_prod_std_max.index
    
    job_res_p_pow_std_min = d_job_res.groupby('order_id')['p_pow_div_std'].min()
    job_res_p_pow_std_min = job_res_p_pow_std_min.to_frame(name='p_pow_div_std_min')
    job_res_p_pow_std_min['order_id'] = job_res_p_pow_std_min.index
    
    job_res_p_pow_std_max = d_job_res.groupby('order_id')['p_pow_div_std'].max()
    job_res_p_pow_std_max = job_res_p_pow_std_max.to_frame(name='p_pow_div_std_max')
    job_res_p_pow_std_max['order_id'] = job_res_p_pow_std_max.index
    #-----------------------------------------------------------------------------------
    
    sigle_job_res_prod = sigle_job_res_prod.merge(sigle_job_res_prod_min,on='order_id',how='left')
    sigle_job_res_prod = sigle_job_res_prod.merge(sigle_job_res_prod_max,on='order_id',how='left')
    
    sigle_job_res_prod = sigle_job_res_prod.merge(job_rese_prod_std_min,on='order_id',how='left')
    sigle_job_res_prod = sigle_job_res_prod.merge(job_rese_prod_std_max,on='order_id',how='left')

    sigle_job_res_prod = sigle_job_res_prod.merge(job_res_p_pow_std_min,on='order_id',how='left')
    sigle_job_res_prod = sigle_job_res_prod.merge(job_res_p_pow_std_max,on='order_id',how='left')
    
    sigle_job_res_prod = sigle_job_res_prod.round(3)
    #d_job_res.to_csv(r'E:\jupyter\CTM\data_processed\resorce\%s.csv'%day,index=False)
    print(day,end=' ')
    sigle_job_res_prod.to_csv(r'E:\jupyter\CTM\data_processed\resorce\p_prod_%s.csv'%day,index=False)

181105 181106 181107 181108 181109 181110 181111 

### 调试代码

In [31]:
sigle_job_res_prod[sigle_job_res_prod['p_pow_div_std_max'].isnull()].shape

(251015, 8)

In [53]:
res['v_day'].apply(lambda x:x.replace('\x1c','')).unique()

array(['180716', '180717', '180718', '180719', '180720', '180721',
       '180722', '180723', '180724', '180725', '180726', '180727',
       '180728', '180729', '180730', '180731', '180801', '180802',
       '180803', '180804', '180805', '180806', '180807', '180808',
       '180809', '180810', '180811', '180812', '180813', '180814',
       '180815', '180816', '180817', '180818', '180819', '180820',
       '180821', '180822', '180823', '180824', '180825', '180826',
       '180827', '180828', '180829', '180830', '180831', '180901',
       '180902', '180903', '180904', '180905', '180906', '180907',
       '180908', '180909', '180910', '180911', '180912', '180913',
       '180914', '180915', '180916', '180917', '180918', '180919',
       '180920', '180921', '180922', '180923', '180924', '180925',
       '180926', '180927', '180928', '180929', '180930', '181001',
       '181002', '181003', '181004', '181005', '181006', '181007',
       '181008', '181009', '181010', '181011', '181012', '1810

In [49]:
day

'181107'

In [24]:
d_job_res[d_job_res['']].head()

,order_id,quant,v_date,max_res,res_key,avg_start_time,res_quat_sum,res_avg_start_std
0,2udb21,8,180722,NaN,1QTY_DXP_SVR_PRCS_WEP@,142.183,631247,369.057148
1,2udm21,1,180722,150.0,1QTY_DXP_DB_PROC_DXPMNG,66.733,58053,353.973117
2,2udot1,2,180722,NaN,1QTY_DXP_SVR_PRCS_WEP@,340.133,631247,369.057148
3,2udos1,7,180722,NaN,1QTY_DXP_SVR_PRCS_WEP@,336.283,631247,369.057148
4,2udrj1,4,180722,NaN,1QTY_DXP_SVR_PRCS_WEP@,1051.033,631247,369.057148


### 以下为调试代码

In [55]:
d_job_res[d_job_res.max_res_y>d_job_res.res_sum].head()

,order_id,resource,quant,v_date,server,max_res_x,max_res_y,res_key,res_sum
374,44r8t,QTY_TDDA_U_JOB_CNT,1,180603,1,NaN,9999.0,QTY_TDDA_U_JOB_CNT1,282
379,44r8r,QTY_TDDA_U_JOB_CNT,1,180603,1,NaN,9999.0,QTY_TDDA_U_JOB_CNT1,282
618,4513k,QTY_DXP_DB_READ_CMB05,1,180603,1,NaN,2000.0,QTY_DXP_DB_READ_CMB051,55
619,4516b,QTY_DXP_DB_READ_CMB05,1,180603,1,NaN,2000.0,QTY_DXP_DB_READ_CMB051,55
930,44yvr,QTY_DXP_SVR_READ_RC3,1,180603,1,NaN,3.0,QTY_DXP_SVR_READ_RC31,2


In [56]:
d_job_res[d_job_res.res_sum==0]

,order_id,resource,quant,v_date,server,max_res_x,max_res_y,res_key,res_sum


In [12]:
d_job_res.head()

,order_id,quant,v_date,max_res,res_key,res_quat_sum,p,p_pow
0,44qk2,1,180603,60.0,1QTY_DXP_JOB_LDELS_WEP,9388,0.006391,0.639114
1,44qmt,1,180603,9.0,1QTY_DXP_DB_WRITE_RDB1,234,0.038462,3.846154
2,40ub1,1,180603,80.0,1QTY_DXP_JOB_EXM_WEP,13797,0.005798,0.579836
3,44qmr,1,180603,9.0,1QTY_DXP_DB_WRITE_RDB1,234,0.038462,3.846154
4,44qpi,1,180603,10.0,1QTY_DXP_DB_WRITE_RWC1,167,0.059880,5.988024


In [10]:
d_job_res.shape

(481064, 8)

In [11]:
d_job_res['p_pow'] = d_job_res['p_pow'].apply(lambda x:x*100)

In [27]:
day = '180603'   #调试用

d_res = res[res.v_day == int(day)]


d_job_res = pd.read_csv(r'E:\jupyter\CTM\data_src\resorce\job_res_num_%s.csv'%day)
d_job_res.drop_duplicates(subset=['order_id','resource','server'],keep='last',inplace=True)


d_job_res = d_job_res.merge(d_res[['resource','server','max_res']],on =['resource','server'],how='left')

#构建唯一键
d_job_res['server'] = d_job_res['server'].apply(str)
d_job_res['res_key'] = d_job_res[['resource','server']].apply(lambda x:x[1]+x[0],axis=1)
d_job_res['order_id'] = d_job_res[['order_id','server']].apply(lambda x:x[0]+x[1],axis=1)

#计算同一资源，有多少其他作业会争抢
res_groupby = d_job_res.groupby(['res_key']).quant.sum()
res_groupby = res_groupby.to_frame(name='res_quat_sum')
res_groupby['res_key'] = res_groupby.index

#合并到同一个表里。
d_job_res = d_job_res.merge(res_groupby,on = 'res_key',how='left')

del d_job_res['resource']
del d_job_res['server']

d_job_res['p'] = d_job_res[['max_res','res_quat_sum']].apply(lambda x:x[0]/x[1] ,axis=1)

d_job_res['p_pow'] = d_job_res[['p','quant']].apply(lambda x:my_pow(x[0],x[1]),axis=1)

#将每个作业的所有资源概率累乘
sigle_job_res_prod = d_job_res.groupby('order_id')['p_pow'].prod()
sigle_job_res_prod = sigle_job_res_prod.to_frame(name='p_prod')
sigle_job_res_prod['order_id'] = sigle_job_res_prod.index

sigle_job_res_prod['p_prod'] = sigle_job_res_prod['p_prod'].apply(lambda x:x*100)

sigle_job_res_prod = sigle_job_res_prod.round(3)
#d_job_res.to_csv(r'E:\jupyter\CTM\data_processed\resorce\%s.csv'%day,index=False)
print(day,end=' ')
sigle_job_res_prod.to_csv(r'E:\jupyter\CTM\data_processed\resorce\p_prod_%s.csv'%day,index=False)

180603 

In [22]:
sigle_job_res_prod = d_job_res.groupby('order_id')['p_pow'].prod()

In [25]:


print(sigle_job_res_prod.head())

         p_prod order_id
order_id                
11bxr3    2.008   11bxr3
11bxs3    2.008   11bxs3
11ndk3    2.008   11ndk3
11nmv3        0   11nmv3
11nmy3        0   11nmy3


In [26]:
sigle_job_res_prod.to_csv(r'E:\jupyter\CTM\data_processed\resorce\p_prod_%s.csv'%day,index=False)

In [19]:
d_job_res.head()

,order_id,resource,quant,v_date,server,max_res
0,5lyiq,QTY_DXP_SVR_WRITE_FAP,1,180618,1,11.0
1,5lwo1,QTY_TDDA_M_JOB_CNT,1,180618,1,9999.0
2,5lyle,QTY_DXP_SVR_WRITE_FAP,1,180618,1,11.0
3,5lwqp,QTY_TDDA_M_JOB_CNT,1,180618,1,9999.0
4,5lwtd,QTY_TDDA_M_JOB_CNT,1,180618,1,9999.0
